# Importing libraries and loading data

In [16]:
import tensorflow as tf
import xgboost as xgb
import xgboost
import tensorflow
import numpy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras import regularizers
from sklearn.svm import SVR
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras import initializers
from datetime import datetime as dt
from utils import *
from sklearn.ensemble import RandomForestRegressor
from features_engineering import TIME_SERIES_FEATURES_ENGINEERING
import sklearn
start=dt.now()

# Process weathertypes, remove empty windspeed and store (ignore if it's already done)

In [17]:
#If you want to generate the one_hot file uncomment the last line of this cell
def update_one_hot_data(combined_path,weather_combined_path):
    if not check_if_the_tsv_hot_file_already_has_processed_columns(weather_combined_path):        
        
        df = pd.read_csv(combined_path,
                         sep='\t',
                         header=0) 
        weather_unique_combinations = df.weather_condition.unique()

        #Get list of all unique weather types
        types = []
        for el in weather_unique_combinations:
            for wc in el.split('.'):
                if(wc != ''):            
                    types.append(wc.strip())
        true_unique =  set(types)

        #Add columns with default value 0 for all unique weather types
        for unique_weather_type in true_unique:
            df[unique_weather_type]=0

        #Loop over all records and set value to 1 for their corresponding weather_types
        for index, row in df.iterrows():
            row_types = []
            for wc in row["weather_condition"].split('.'):
                if(wc != ''):            
                    row_types.append(wc.strip())
            for t in row_types:
                df.at[index,t]=1


        #remove empty windspeeds
        df = df[df.wind_speed.apply(lambda x: str(x).isnumeric())]

        #remove original weather_condition column and store, to avoid rerun
        df =  df.drop(columns="weather_condition")
        df.to_csv(weather_combined_path,
                  sep='\t',
                  index=False,
                  header=True) 
        TIME_SERIES_FEATURES_ENGINEERING(weather_combined_path)
    else:
        print("File already processed.")
        
combined_path = ".././data/combined_data.tsv"
weather_combined_path=".././data/combined_one_hot_data.tsv"
#update_one_hot_data(combined_path,weather_combined_path)

# Loadin data with one_hot weather types

In [18]:
def select_data_from_lat_csv(data,latitude):

    return data[data['latitude']==latitude]
def obtain_latitudes_list(latitude_data):
    return latitude_data.unique()
weather_combined_path=".././data/combined_one_hot_data.tsv"
df = pd.read_csv(weather_combined_path,
                 sep='\t',
                 header=0)
print(df.shape)
lat_list=obtain_latitudes_list(df.latitude)
pole=5
# df=select_data_from_lat_csv(df,lat_list[pole])
# print("Selecting data for bikes station on latitude "+str(lat_list[pole]))
#print(lat_list)
print(df.shape)

#choose one station to test on
#df=df[df['device_name']=='CB1143']
#print(df)

(101461, 48)
(101461, 48)


# Remove outliers (bike_count)

In [19]:
def remove_outlier(df_in, col_name):
    """Removes all outliers on a specific column from a given dataframe.

    Args:
        df_in (pandas.DataFrame): Iput pandas dataframe containing outliers
        col_name (str): Column name on which to search outliers

    Returns:
        pandas.DataFrame: DataFrame without outliers
    """         
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1  # Interquartile range
    fence_low = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    return df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)] 
start_size=df.shape[0]
df = remove_outlier(df, "bike_count")
print(df.shape)
print("We have removed "+str(start_size-df.shape[0])+" outliers tuples (which is "+str((start_size-df.shape[0])/start_size*100)+" % of total).")

(94342, 48)
We have removed 7119 outliers tuples (which is 7.016489094331813 % of total).


# Feature selection

In [20]:
def remove_features(df):
    return df.drop(columns=[        
    'device_name',
    'timestamp_until',
    'bike_avg_speed',
    'weather_timestamp',
    'wind_direction',
    'wind_speed',
    'barometer',
    'visibility',
    'Ice fog',
    'Thundershowers',
    'Sprinkles',
    'Broken clouds',
    'Rain showers',
    'Snow flurries',
    'Light fog',
    'Sleet',
    'Cloudy',
    'Quite cool'    
])
def remove__weather_features(df):
    return df.drop(columns=[        
    'temperature',
    'humidity',
    'Scattered showers',
    'Low clouds',
    'Snow',
    'Snow showers',
    'Thunderstorms',
    'Partly sunny',
    'Light freezing rain',
    'Sunny',
    'Light rain',
    'Freezing rain',
    'Light snow',
    'Passing clouds',
    'Fog',
    'Cool',
    'Partly cloudy',
    'Haze',
    'Hail',
    'Scattered clouds',
    'Drizzle',
    'Clear',
    'Rain',
    'Chilly'    
        
])
def remove__all_features_for_ARIMA_Model(df):
    return df.drop(columns=[        
    'temperature',
    'latitude',
    'longitude',
    'time_window',
    'time_day_of_week_window',
    'humidity',
    'Scattered showers',
    'Low clouds',
    'Snow',
    'Snow showers',
    'Thunderstorms',
    'Partly sunny',
    'Light freezing rain',
    'Sunny',
    'Light rain',
    'Freezing rain',
    'Light snow',
    'Passing clouds',
    'Fog',
    'Cool',
    'Partly cloudy',
    'Haze',
    'Hail',
    'Scattered clouds',
    'Drizzle',
    'Clear',
    'Rain',
    'Chilly' 
        
])

df=remove_features(df)
df=remove__all_features_for_ARIMA_Model(df)
#df=remove__weather_features(df)
print(df.head())
print(df.columns)
print(df.shape)

   timestamp_from  bike_count
0      1544095800           3
1      1544096700           9
2      1544097600          10
3      1544098500          10
4      1544099400           3
Index(['timestamp_from', 'bike_count'], dtype='object')
(94342, 2)


# Creating ARIMA model

In [ ]:
def ARIMA_bikes():
    from pandas import read_csv
    from pandas import datetime
    from matplotlib import pyplot
    from statsmodels.tsa.arima_model import ARIMA
    from sklearn.metrics import mean_squared_error
    from sklearn.metrics import mean_absolute_error
    import numpy as np
    import math

    def parser(x):
        # print(datetime.fromtimestamp(int(x)))
        # print(type(datetime.fromtimestamp(int(x))))
        # input()
        return datetime.fromtimestamp(int(x))

    series = read_csv('point_50.82448.tsv',sep='\t', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
    print(df)
    print(type(df))
    print(series)
    print(type(series))
    input()
    # print(series.head())
    # series.plot()
    # pyplot.show()
    # print(series.values)
    # print(type(series.values))
    # input()
    count=0
    X = series.values
    size = int(len(X) * 0.998)
    train, test = X[0:size], X[size:len(X)]
    history = [x for x in train]
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=(5,1,0))
        model_fit = model.fit(disp=0)
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        obs = test[t]
        history.append(obs)
        print(count)
        count+=1
        print('predicted=%f, expected=%f' % (yhat, obs))
    # del predictions[0]	
    # predictions.append(np.array([test[-1]]))
    error = mean_squared_error(test, predictions)
    mae=mean_absolute_error(test,predictions)
    print('Test MAE : %.3f' % mae)
    print('Test MSE : %.3f' % error)
    print("Test RMSE:%.3f "%math.sqrt(error))
    # plot
    print(type(predictions))
    pyplot.plot(test)
    pyplot.plot(predictions, color='red')
    pyplot.show()
ARIMA_bikes()

        timestamp_from  bike_count
0           1544095800           3
1           1544096700           9
2           1544097600          10
3           1544098500          10
4           1544099400           3
5           1544445900          11
6           1544446800          16
7           1544447700           9
8           1544448600          16
9           1544449500           5
10          1544510700          14
12          1544512500          22
13          1544513400          15
14          1544514300          14
15          1544515200          17
16          1544516100           6
17          1544517000           4
18          1544517900           3
19          1544518800           8
20          1544519700          10
21          1544520600           9
22          1544521500          11
23          1544522400           9
24          1544523300          10
25          1544524200          10
26          1544525100          13
27          1544526000           9
28          15445269

In [ ]:
 print(dt.now()-start)